In [2]:
from SRCNN_different_specs import RunSRCNN, SRCNN
from custom_image_dataset import CustomImageDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import torch
from torch import nn

In [3]:
DATASET_ROOT="./datasets/"
DATASET_NAME = "Set14"

In [4]:
%%time
transform_x = transforms.Compose([
    transforms.Resize((11, 11), interpolation=Image.BICUBIC),
    transforms.Resize((33, 33), interpolation=Image.BICUBIC)
])

transform_y = transforms.Compose([
    transforms.CenterCrop((33, 33))
])

# Set up the data loaders
train_data_set = CustomImageDataset(
    img_dir=f"{DATASET_ROOT}/{DATASET_NAME}_train", transform=transform_x, target_transform=transform_y)
validation_data_set = CustomImageDataset(
    img_dir=f"{DATASET_ROOT}/{DATASET_NAME}_validation", transform=transform_x, target_transform=transform_y)

# Report split sizes
print('Training set has {} instances'.format(len(train_data_set)))
print('Validation set has {} instances'.format(len(validation_data_set)))

/Users/pedrovelozo/anaconda3/lib/python3.10/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
100%|██████████| 14/14 [00:00<00:00, 220.96it/s]

Training set has 1085 instances
Validation set has 280 instances
CPU times: user 132 ms, sys: 126 ms, total: 259 ms
Wall time: 554 ms


In [5]:
%%time

# https://towardsdatascience.com/7-tips-for-squeezing-maximum-performance-from-pytorch-ca4a40951259
pin_memory = False
batch_size = round(0.40*(2**13))

train_dataloader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True, pin_memory=pin_memory) # TODO: aumentar batch_size p/ 2^12
validation_dataloader = DataLoader(validation_data_set, batch_size=batch_size, pin_memory=pin_memory)

CPU times: user 8.54 ms, sys: 10 ms, total: 18.6 ms
Wall time: 17.9 ms


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 1e-3

model_f2_5 = SRCNN(f2=5).to(device)
optimizer_f2_5 = torch.optim.Adam(model_f2_5.parameters(), lr=lr)
scheduler_f2_5 = torch.optim.lr_scheduler.LinearLR(
    optimizer_f2_5,
    start_factor=1.0,
    end_factor=0.01,
    total_iters=10)

model_f2_1 = SRCNN(f2=1).to(device)
optimizer_f2_1 = torch.optim.Adam(model_f2_1.parameters(), lr=lr)
scheduler_f2_1 = torch.optim.lr_scheduler.LinearLR(
    optimizer_f2_1,
    start_factor=1.0,
    end_factor=0.01,
    total_iters=10)


In [8]:
run_srccn = RunSRCNN()

In [9]:
run_srccn.train(
    model=model_f2_1,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader,
    optimizer=optimizer_f2_1,
    epochs=10,
    scheduler=scheduler_f2_1,
    loss_fn=nn.MSELoss(),
)


epoch 0
-------------------------------
Training on 1085 samples...
Batch size: 3277
Number of batches: 1


  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: NNPACK SpatialConvolution_updateOutput failed